# Example: Monitoring Reforestation Projects
This notebook demonstrates how to work with **VerdeSat** services on a real-world dataset. The GeoJSON below represents three small reforestation projects in Chiapas, Mexico. We'll compute NDVI time series, land-cover metrics, a B-Score, and validate using citizen‑science observations.

In [26]:
geojson_path = "examples/reforestation_plots.geojson"

In [ ]:
from verdesat.geo.aoi import AOI
from verdesat.analytics.timeseries import TimeSeries
from verdesat.analytics.engine import AnalyticsEngine
from verdesat.analytics.stats import compute_summary_stats
from verdesat.visualization.visualizer import Visualizer

In [28]:
aois = AOI.from_geojson(geojson_path, id_col='ID2')
for aoi in aois:
    print(aoi.static_props.get('id'), aoi.geometry.area)

None 2.2290295417198608e-06
None 2.0710699275829863e-06
None 8.769081313212721e-06


In [ ]:
import pandas as pd

# Load precomputed NDVI time series
ts_df = pd.read_csv('examples/reforestation_ts.csv', parse_dates=['date'])
ts_df.head()

In [ ]:
import geopandas as gpd
import folium

# Visualise AOIs on an interactive map
gdf = gpd.read_file(geojson_path)
center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=15)
folium.GeoJson(gdf, name='AOIs').add_to(m)
m.save('examples/map.html')
m

In [32]:
from verdesat.analytics.timeseries import TimeSeries
from verdesat.analytics.stats import compute_summary_stats
from verdesat.visualization.visualizer import Visualizer
import pandas as pd, os

ts = TimeSeries.from_dataframe(ts_df, index='ndvi')
ts_monthly = ts.aggregate('ME').fill_gaps()
decomposed = ts_monthly.decompose(period=12)

os.makedirs('examples/decomp', exist_ok=True)
for pid, res in decomposed.items():
    pd.DataFrame({
        'date': res.observed.index,
        'observed': res.observed.values,
        'trend': res.trend.values,
        'seasonal': res.seasonal.values,
        'resid': res.resid.values,
    }).to_csv(f'examples/decomp/{pid}_decomposition.csv', index=False)

trend = AnalyticsEngine.compute_trend(ts_monthly)
stats = compute_summary_stats('examples/reforestation_ts.csv', decomp_dir='examples/decomp', period=12)
stats.to_dataframe()

,Site ID,Start Date,End Date,Num Periods,% Gapfilled,Mean NDVI,Median NDVI,Min NDVI,Max NDVI,Std NDVI,Sen's Slope (NDVI/yr),Trend ΔNDVI,Mann–Kendall p-value,Seasonal Amplitude,Peak Month,Residual RMS
0,1,2019-01,2024-12,2178,NaN,0.761404,0.765169,0.541452,0.893406,0.054653,-0.002310,-0.005403,0.055700,0.108246,2019-08,0.033910
1,2,2019-01,2024-12,2178,NaN,0.625843,0.628658,0.400309,0.847535,0.107592,-0.011532,-0.017681,0.000002,0.238489,2019-09,0.024385
2,3,2019-01,2024-12,2178,NaN,0.685285,0.683405,0.540685,0.806831,0.064848,-0.000279,0.005142,0.858266,0.145770,2019-07,0.017901


In [33]:
viz = Visualizer()
viz.plot_time_series(ts_monthly.df, 'mean_ndvi', 'examples/ndvi_timeseries.png', agg_freq='ME')
viz.plot_timeseries_html(ts_monthly.df, 'mean_ndvi', 'examples/ndvi_timeseries.html', agg_freq='ME')
for pid, res in decomposed.items():
    viz.plot_decomposition(res, f'examples/decomp/{pid}_decomp.png')
trend.to_dataframe()

,id,date,trend
0,1,2019-01-31,0.756737
1,1,2019-02-28,0.756878
2,1,2019-03-31,0.757035
3,1,2019-04-30,0.757186
4,1,2019-05-31,0.757342
...,...,...,...
211,3,2024-08-31,0.713189
212,3,2024-09-30,0.713713
213,3,2024-10-31,0.714255
214,3,2024-11-30,0.714779


The interactive map is saved as `examples/map.html`. Open this file in a browser to explore the project locations.

In [ ]:
csv_path = Path('examples/bscore.csv')
if not csv_path.exists():
    df_scores = compute_bscores(geojson_path, year=2021, output=str(csv_path))
else:
    df_scores = pd.read_csv(csv_path)
df_stats = stats.to_dataframe()
df = df_stats.merge(df_scores, left_on='Site ID', right_on='id').set_index('Site ID')
df[['bscore', "Sen's Slope (NDVI/yr)"]].plot.bar(figsize=(8,4))df.head()

In [ ]:
# All plots have been written to the examples directory.

### What do the metrics tell us?

* **Mean NDVI** represents the average greenness of each plot. Higher values indicate denser, healthier vegetation.
* **Seasonal amplitude** captures the difference between peak and trough NDVI values and highlights intra‑annual variability.
* **Sen's slope** estimates the yearly change in NDVI after removing seasonal effects. Positive values mean the site is greening over time.
* **Residual RMS** indicates short‑term noise left after decomposition.

For this dataset site 1 has the highest mean NDVI, while site 3 shows the smallest decline (highest Sen's slope). Together the metrics suggest that plot 1 is currently the greenest but plot 3 is stabilising the best.